In [ ]:
import matplotlib.pyplot as plt #load plotting library
import numpy as np #load numerical library
import sympy #load symbolic library

# Lab 8: Demography

In lecture we discussed how we can apply what we know about linear multivariate models to populations structured by stage or age (demography). There we looked at two examples: a stage-structured model of North Atlantic right whales and an age-structured model of stickleback. In both cases we assumed we had good estimates of the entries in the transition matrix. Here we'll see how to build a transition matrix from data, and how to analyze a larger transition matrix with the help of Python.

## The demography of Canadians

Following section 10.6 in the text, here we'll use data from past censuses to build a Leslie matrix and analyze how the demography of Canadians is expected to change. 

Because the census takes place every 5 years, instead of tracking individuals of each age we'll track individuals in 5-year incremements: 0-4 years, 5-9 years, .... This ensures all individuals in a given age class will either die or move to the next age class, i.e., we get a Leslie matrix. We'll also group all individuals over 85 into the final age class, which means that we have a "modified Leslie matrix" with a non-zero survival probability of staying in the final age class from one time steo to the next. Because these individuals in the final age class do not reproduce, this actually doesn't complicate things much.

We'll track just the number of females in each age class for simplicity (a classic approach when the number of offspring produced does not depend on the number of males), so we'll have to multiply the number of offspring by the sex ratio (fraction female) to get the expected number of female offspring.

First, let's record how many females were in each age class in 1991 (which we'll take to be a representative time) 

In [ ]:
age = ['%d-%d' %(i,i+4) for i in range(0,85,5)] + ['85+'] #list of age classes (just to make nicer plots)
number = np.array([945400, 909700, 901500, 895900, 976800, 1168800, 1221800, 1133000, 1031500, 807500, 656600, 610100, 599400, 571800, 461700, 362600, 237900, 197100]) #number of females in each age class

In [ ]:
fig,ax = plt.subplots() #set up figure
ax.bar(age,number) #plot data
ax.set_xlabel('age class')
ax.set_ylabel('number of females')
ax.tick_params(axis='x', rotation=60) #rotate x labels for clarity
plt.show()

One other piece of data that we'll need is the number of females of each age class that died that year.

In [ ]:
deaths = np.array([1344, 155, 152, 345, 361, 556, 681, 939, 1337, 1659, 2142, 3275, 5032, 7763, 9757, 12949, 14261, 27415])

In [ ]:
fig,ax = plt.subplots() #set up figure
ax.bar(age, deaths) #plot data
ax.set_xlabel('age class')
ax.set_ylabel('number of females that died')
ax.tick_params(axis='x', rotation=60) #rotate x labels for clarity
plt.show()

And the last bit of data we'll use is the number of births by females in each age class in 1991.

In [ ]:
births = np.array([0, 0, 265, 24180, 80723, 150024, 107560, 33107, 4124, 138, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
fig,ax = plt.subplots() #set up figure
ax.bar(age, births) #plot data
ax.set_xlabel('age class')
ax.set_ylabel('number of offspring produced')
ax.tick_params(axis='x', rotation=60) #rotate x labels for clarity
plt.show()

Our first goal is to build a transition matrix, $\mathbf{L} = \begin{pmatrix} m_1 & m_2 & \cdots & m_d \\ p_1 & 0 & \cdots & 0 \\ 0 & p_2 & \cdots & 0 \\ \vdots & \vdots & \vdots & \vdots \\ 0 & 0 & \cdots & p_d \end{pmatrix}$, from this data, such that the number of females in age class, written as a vector $\vec{n}$, changes according to the linear recursion $\vec{n}(t+5) = \mathbf{L} \vec{n}(t)$, where $t$ is measured in years -- remember that since the the age classes are 5 years long we will predict the population size in five year increments. To do this we need to know the survival fractions, $p_i$, and the fecundities, $m_i$.

Once the transition matrix is complete we can then use what we've learned to predict how the population of Canada will change over time!

## Questions

**Q1.** [2 points] Use the total number of females in an age class, $n_i$, and the number of female deaths in that age class that year, $d_i$, to calculate the fraction of females that survive that entire age class, $p_i = (1 - d_i/n_i)^5$, i.e., they do not die in all five years that they are in that age class. Calculate the survival fraction, $p_i$, for each age class, making a list called ```p```. Make a bar plot to better see the pattern.

**Q2.** [2 points] Use the total number of females in an age class, $n_i$, the number of births per year in that age class, $b_i$, and the fraction of offspring that are female, $f = 0.4867$, to calculate the fecundity of each class over 5 year increments, $m_i = 5f b_i/n_i$. Calculate the $m_i$'s, making a list called ```m```. Make a bar plot to see what the data looks like.

OK, now we have all the elements of our Leslie matrix. Putting things together is a bit tricky, but here is one way to do it (try writing a script yourself if you'd like -- I'm sure there are better ways to do it!)

In [ ]:
# here are the survival fractions and fecundities, in case you had trouble calculating them above
p = [0.99291208, 0.99914836, 0.99915724, 0.99807604, 0.99815349,
       0.99762375, 0.99721623, 0.995863  , 0.99353593, 0.98976968,
       0.98379478, 0.97344675, 0.95872357, 0.93393621, 0.8987087 ,
       0.83374813, 0.73411737, 0.47291615]
m = [0.        , 0.        , 0.00071534, 0.06567924, 0.20110506,
       0.31235746, 0.21423086, 0.07110846, 0.00972928, 0.00041588,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ]

# make the matrix
d = len(age) #number of age classes (dimension)
L = np.zeros((d,d)) #start with a matrix of all zeros
L[0] = m #put fecundities in the first row of L
for i,pi in enumerate(p[:-1]): #for all but the final age class...
    L[i+1,i] = pi #put the survival fraction in row i+1, column i of L
L[-1,-1] = p[-1] #add the survival fraction for the final class as the bottom right entry in L

#print(L) #take a look at the matrix if you want

# here is another way to visualize a matrix, as a heatmap -- you can see the fecundities in the first row and the survival fractions just below the diagonal 
fig,ax = plt.subplots() #set up figure
im = ax.imshow(L) #plot matrix
plt.colorbar(im, ax=ax) #add color scale
ax.set_xticks([i for i in range(len(age))], age)
ax.set_yticks([i for i in range(len(age))], age)
ax.tick_params(axis='x', rotation=60) #rotate x labels for clarity
ax.set_xlabel('transitioning from')
ax.set_ylabel('transitioning to')
plt.show()

Now that we have the Leslie matrix, we can calculate the eigenvalues and right eigenvectors with

In [ ]:
evs = np.linalg.eig(L)

The eigenvalues are in the first entry

In [ ]:
evs[0]

(the "j" is our $i = \sqrt{-1}$)

The right eigenvectors are the columns of the second entry, so to get the right eigenvector associated with the $i^{th}$ eigenvalue you would run ```evs[1][:,i-1]```. For example, if we wanted the right eigenvector assoicated with the first eigenvalue we'd run

In [ ]:
evs[1][:,0]

as these are all real numbers (ie, the imaginary parts, the number multiplied by j, are all 0) we can simplify by just looking at the real part

In [ ]:
np.real(evs[1][:,0])

And finally, if you wanted to scale this vector so that it summed to one, you just divide each entry by the sum

In [ ]:
v = np.real(evs[1][:,0])
v/np.sum(v)

These are the pieces we need to answer the following questions.

**Q3.** [2 points] Now, what is the expected long-term population growth rate given this Leslie matrix? (Hint: find the leading eigenvalue numerically using ```np.abs()```, and ```np.max()```). Is the population expected to grow or decline? Given that we know the Canadian population has grown since 1991, what are we missing? 

Note that you should get the (approximately) same answer using the Euler-Lotka equation, $1 = \sum_{i=1}^{d}\frac{l_i m_i}{\lambda^i}$, which we can do without having to construct the actual Leslie matrix.

In [ ]:
l = [np.prod(p[:i]) for i in range(len(p))] #probability of surviving to age class i
sympy.var('x') #this will be our lambda that we want to solve for
rhs = np.sum([li*mi/x**(i+1) for i,(li,mi) in enumerate(zip(l,m))]) #the right hand side of the Euler-Lotka equation
roots = sympy.roots(x**10*(rhs - 1), x, multiple=True) #numerically find the solutions x of the Euler-Lotka equation
max(np.abs(roots)) #leading eigenvalue

**Q4.** [2 points] Use the Leslie matrix to iterate the population forward to 2021, $\vec{n}(t) = \mathbf{L}^t\vec{n}(0)$. Remember that we are starting in year 1991 and counting in 5 year increments! Make a bar chart of the number of females in each age class in 2021. What age class do we expect to be most frequent?

**Q5.** [2 points] What is the predicted frequency of each age class in the long-run? (Hint: find the right eigenvector associated with the leading eigenvalue, remove the imaginary parts (which are zero), and scale the entries so that the sum equals 1). Make a bar chart to see this visually. What age class do we expect to be most frequent?